# Strands Agent SDK

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))

In [3]:
import os
from strands import Agent
from strands.models import BedrockModel
from src.utils.bedrock import bedrock_info
from botocore.config import Config

## 1. Utils

### 1.1 Get llm model by inference type

In [4]:
def get_llm_by_type(llm_type, cache_type=None, enable_reasoning=False):
    """
    Get LLM instance by type. Returns cached instance if available.
    """
    if llm_type == "reasoning":
        
        ## BedrockModel params: https://strandsagents.com/latest/api-reference/models/?h=bedrockmodel#strands.models.bedrock.BedrockModel
        llm = BedrockModel(
            model_id=bedrock_info.get_model_id(model_name="Claude-V3-7-Sonnet-CRI"),
            streaming=True,
            max_tokens=8192*5,
            stop_sequencesb=["\n\nHuman"],
            temperature=1 if enable_reasoning else 0.01, 
            additional_request_fields={
                "thinking": {
                    "type": "enabled" if enable_reasoning else "disabled", 
                    **({"budget_tokens": 8192} if enable_reasoning else {}),
                }
            },
            cache_prompt=cache_type, # None/ephemeral/defalut
            #cache_tools: Cache point type for tools
            boto_client_config=Config(
                read_timeout=900,
                connect_timeout=900,
                retries=dict(max_attempts=50, mode="adaptive"),
            )
        )
        
    elif llm_type == "basic":
        ## BedrockModel params: https://strandsagents.com/latest/api-reference/models/?h=bedrockmodel#strands.models.bedrock.BedrockModel
        llm = BedrockModel(
            model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-V-2-Sonnet-CRI"),
            streaming=True,
            max_tokens=8192,
            stop_sequencesb=["\n\nHuman"],
            temperature=0.01,
            cache_prompt=cache_type, # None/ephemeral/defalut
            #cache_tools: Cache point type for tools
            boto_client_config=Config(
                read_timeout=900,
                connect_timeout=900,
                retries=dict(max_attempts=50, mode="standard"),
            )
        )

    else:
        raise ValueError(f"Unknown LLM type: {llm_type}")
        
    return llm

### 1.2 Create agent

In [5]:
from datetime import datetime
from src.utils.bedrock import bedrock_utils

In [21]:
class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

def apply_prompt_template(prompt_name: str, prompt_cache=False, cache_type="default") -> list:
    
    system_prompts = open(os.path.join("./prompts", f"{prompt_name}.md")).read()    
    context = {"CURRENT_TIME": datetime.now().strftime("%a %b %d %Y %H:%M:%S %z")}
    system_prompts = system_prompts.format(**context)
        
    return system_prompts

In [22]:
def get_agent(**kwargs):

    agent_name = kwargs["agent_name"]
    tools = kwargs.get("tools", None)
    streaming = kwargs.get("streaming", True)

    agent_llm_map = kwargs["agent_llm_map"]
    agent_prompt_cache_map = kwargs["agent_prompt_cache_map"]
    
    if "reasoning" in agent_llm_map[agent_name]: enable_reasoning = True
    else: enable_reasoning = False

    prompt_cache, cache_type = agent_prompt_cache_map[agent_name]
    if prompt_cache: print(f"{Colors.GREEN}{agent_name.upper()} - Prompt Cache Enabled{Colors.END}")
    else: print(f"{Colors.GREEN}{agent_name.upper()} - Prompt Cache Disabled{Colors.END}")

    system_prompts = apply_prompt_template(agent_name)
    llm = get_llm_by_type(agent_llm_map[agent_name], cache_type, enable_reasoning)    
    llm.config["streaming"] = streaming

    agent = Agent(
        model=llm,
        system_prompt=system_prompts,
        tools=tools,
        callback_handler=None # async iterator로 대체 하기 때문에 None 설정
    )

    return agent

### 1.3 Response with streaming

In [23]:
import traceback
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [24]:
class ColoredStreamingCallback(StreamingStdOutCallbackHandler):
    COLORS = {
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'red': '\033[91m',
        'purple': '\033[95m',
        'cyan': '\033[96m',
        'white': '\033[97m',
    }
    
    def __init__(self, color='blue'):
        super().__init__()
        self.color_code = self.COLORS.get(color, '\033[94m')
        self.reset_code = '\033[0m'
    
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"{self.color_code}{token}{self.reset_code}", end="", flush=True)

In [51]:
async def process_streaming_response(agent, message):
    callback_reasoning, callback_answer = ColoredStreamingCallback('purple'), ColoredStreamingCallback('white')
    response = {"text": "","reasoning": "", "signature": "", "tool_use": None, "cycle": 0}
    try:
        agent_stream = agent.stream_async(message)
        async for event in agent_stream:
            if "reasoningText" in event:
                response["reasoning"] += event["reasoningText"]
                callback_reasoning.on_llm_new_token(event["reasoningText"])
            elif "reasoning_signature" in event:
                response["signature"] += event["reasoning_signature"]
            elif "data" in event:
                response["text"] += event["data"]
                callback_answer.on_llm_new_token(event["data"])
            elif "current_tool_use" in event and event["current_tool_use"].get("name"):
                response["tool_use"] = event["current_tool_use"]["name"]
                if "event_loop_metrics" in event:
                    if response["cycle"] != event["event_loop_metrics"].cycle_count:
                        response["cycle"] = event["event_loop_metrics"].cycle_count
                        callback_answer.on_llm_new_token(f' \n## Calling tool: {event["current_tool_use"]["name"]} - # Cycle: {event["event_loop_metrics"].cycle_count}\n')
    except Exception as e:
        print(f"Error in streaming response: {e}")
        print(traceback.format_exc())  # Detailed error logging
    
    return agent, response

## 2. Usage

### 2.1 Agent definition

- Agent config

In [52]:
from typing import Literal, Tuple

In [53]:
# Define available LLM types
LLMType = Literal["basic", "reasoning"]
CACHEType = Tuple[bool, Literal["default", "ephemeral"]]

# Define agent-LLM mapping
AGENT_LLM_MAP: dict[str, LLMType] = {"task_agent": "basic"} # "reasoning"
AGENT_PROMPT_CACHE_MAP: dict[bool, CACHEType] = {"task_agent": (False, None)} # (True, "default")

- system prompt

In [54]:
%%writefile ./prompts/task_agent.md
---
CURRENT_TIME: {CURRENT_TIME}
---

You are Bedrock-Manus, a friendly AI assistant developed by the Bedrock-Manus team.
You specialize in handling greetings and small talk.

Overwriting ./prompts/task_agent.md


In [55]:
agent = get_agent(
    agent_name="task_agent",
    streaming=True,
    agent_llm_map=AGENT_LLM_MAP,
    agent_prompt_cache_map=AGENT_PROMPT_CACHE_MAP
)

TASK_AGENT - Prompt Cache Disabled


### 2.2 Execution

In [56]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [57]:
message = "안녕 나는 장동진이야"
agent, response = asyncio.run(process_streaming_response(agent, message))


안녕하세요 장동진님! 만나서 반갑습니다. 오늘 기분은 어떠신가요?

## 3. Tools

In [58]:
from src.tools import python_repl_tool, bash_tool

In [59]:
agent = get_agent(
    agent_name="task_agent",
    tools=[python_repl_tool, bash_tool],
    streaming=True,
    agent_llm_map=AGENT_LLM_MAP,
    agent_prompt_cache_map=AGENT_PROMPT_CACHE_MAP
)

TASK_AGENT - Prompt Cache Disabled


In [60]:
message = "./prompts 디렉토리에 어떤 파일이 있는지 확인해 줄래?"
agent, response = asyncio.run(process_streaming_response(agent, message))

네, './prompts' 디렉토리의 파일 목록을 확인해보겠습니다.


INFO [bash_tool] ===== Executing Bash =====

INFO [bash_tool] ===== Coder - Command: ls -l ./prompts =====

INFO [src.tools.decorators] 
Coder - Tool handle_bash_tool returned:
ls -l ./prompts||total 4
-rw-rw-r-- 1 ubuntu ubuntu 175 Jul 25 04:55 task_agent.md




'./prompts' 디렉토리에는 'task_agent.md' 파일이 하나 있네요. 이 파일의 크기는 175바이트이고, 마지막 수정 시간은 7월 25일 04:55입니다.

In [61]:
message = "Hello world 를 프린팅하는 파이썬 코드를 작성하고 실행시켜 줄래?"
agent, response = asyncio.run(process_streaming_response(agent, message))

네, "Hello world"를 출력하는 간단한 파이썬 코드를 작성하고 실행하겠습니다.


INFO [python_repl_tool] ===== Executing Python code =====

INFO [python_repl_tool] ===== Code execution successful =====

INFO [src.tools.decorators] Coder - Successfully executed:

```python
print("Hello world")
```


INFO [src.tools.decorators] 
Stdout: Hello world



파이썬의 print() 함수를 사용하여 "Hello world"를 성공적으로 출력했습니다.

## 4. built-in utility

In [62]:
from pprint import pprint

### 4.1 Check agent

- Syetem prompt

In [63]:
system_prompt = agent.system_prompt
pprint(system_prompt)

('---\n'
 'CURRENT_TIME: Fri Jul 25 2025 04:55:50 \n'
 '---\n'
 '\n'
 'You are Bedrock-Manus, a friendly AI assistant developed by the '
 'Bedrock-Manus team.\n'
 'You specialize in handling greetings and small talk.\n')


- Message history

In [64]:
agent_messages = agent.messages
pprint(agent_messages)

[{'content': [{'text': './prompts 디렉토리에 어떤 파일이 있는지 확인해 줄래?'}], 'role': 'user'},
 {'content': [{'text': "네, './prompts' 디렉토리의 파일 목록을 확인해보겠습니다."},
              {'toolUse': {'input': {'cmd': 'ls -l ./prompts'},
                           'name': 'bash_tool',
                           'toolUseId': 'tooluse_1guHDYQ2Su-tAXSLOH3chQ'}}],
  'role': 'assistant'},
 {'content': [{'toolResult': {'content': [{'text': 'ls -l ./prompts||total 4\n'
                                                   '-rw-rw-r-- 1 ubuntu ubuntu '
                                                   '175 Jul 25 04:55 '
                                                   'task_agent.md\n'
                                                   '\n'}],
                              'status': 'success',
                              'toolUseId': 'tooluse_1guHDYQ2Su-tAXSLOH3chQ'}}],
  'role': 'user'},
 {'content': [{'text': "'./prompts' 디렉토리에는 'task_agent.md' 파일이 하나 있네요. 이 파일의 "
                       '크기는 175바이트이고, 마지막 수정 시간은 7월 2

- observility

In [65]:
pprint(agent.event_loop_metrics)

EventLoopMetrics(cycle_count=4,
                 tool_metrics={'bash_tool': ToolMetrics(tool={'input': {'cmd': 'ls '
                                                                               '-l '
                                                                               './prompts'},
                                                              'name': 'bash_tool',
                                                              'toolUseId': 'tooluse_1guHDYQ2Su-tAXSLOH3chQ'},
                                                        call_count=1,
                                                        success_count=1,
                                                        error_count=0,
                                                        total_time=0.005025625228881836),
                               'python_repl_tool': ToolMetrics(tool={'input': {'code': 'print("Hello '
                                                                                       'world")'},
      

- Resume

In [66]:
llm_ = BedrockModel(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-7-Sonnet-CRI"),
    streaming=True,
    max_tokens=8192,
    stop_sequencesb=["\n\nHuman"],
    temperature=0.01,
    cache_prompt=None, # None/ephemeral/defalut
    #cache_tools: Cache point type for tools
    boto_client_config=Config(
        read_timeout=900,
        connect_timeout=900,
        retries=dict(max_attempts=50, mode="standard"),
    )
)


agent_ = Agent(
    model=llm_,
    tools=[python_repl_tool, bash_tool],
    system_prompt=system_prompt,
    messages=agent_messages,
    callback_handler=None # async iterator로 대체 하기 때문에 None 설정
)

In [67]:
message = "이어서 대화 하는거 맞니?"
agent, response = asyncio.run(process_streaming_response(agent_, message))

네, 맞습니다! 지금 이어서 대화하고 있습니다. 제가 Bedrock-Manus라는 AI 어시스턴트로서 계속해서 도움을 드리고 있어요. 이전에 "Hello world"를 출력하는 파이썬 코드를 실행해 드렸고, 지금도 계속해서 대화를 이어가고 있습니다. 다른 질문이나 도움이 필요한 것이 있으시면 말씀해 주세요!

### 4.2 [Conversation management](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/conversation-management/?h=conversa)

As conversations grow, managing this context becomes increasingly important for several reasons:

- **Token Limits**: Language models have fixed context windows (maximum tokens they can process)
- **Performance**: Larger contexts require more processing time and resources
- **Relevance**: Older messages may become less relevant to the current conversation
- **Coherence**: Maintaining logical flow and preserving important information


#### 4.2.1. SlidingWindowConversationManager
고정된 수의 최근 메시지를 유지하는 슬라이딩 윈도우 전략을 구현합니다. Agent 클래스에서 기본적으로 사용하는 대화 매니저입니다.

In [68]:
from strands.agent.conversation_manager import SlidingWindowConversationManager

In [69]:
# Create a conversation manager with custom window size
conversation_manager = SlidingWindowConversationManager(
    window_size=3,  # Maximum number of messages to keep
    should_truncate_results=True, # Enable truncating the tool result when a message is too large for the model's context window 
)

In [70]:
agent.conversation_manager = conversation_manager

In [71]:
message = "안녕 나는 장동진이야"
agent, response = asyncio.run(process_streaming_response(agent, message))
print ("\n")
pprint (agent.messages)

안녕하세요, 장동진님! 만나서 반갑습니다. 저는 Bedrock-Manus라는 AI 어시스턴트입니다. 오늘 어떻게 도와드릴까요? 질문이 있으시거나 도움이 필요한 일이 있으시면 말씀해 주세요.

[{'content': [{'text': './prompts 디렉토리에 어떤 파일이 있는지 확인해 줄래?'}], 'role': 'user'},
 {'content': [{'text': "네, './prompts' 디렉토리의 파일 목록을 확인해보겠습니다."},
              {'toolUse': {'input': {'cmd': 'ls -l ./prompts'},
                           'name': 'bash_tool',
                           'toolUseId': 'tooluse_1guHDYQ2Su-tAXSLOH3chQ'}}],
  'role': 'assistant'},
 {'content': [{'toolResult': {'content': [{'text': 'ls -l ./prompts||total 4\n'
                                                   '-rw-rw-r-- 1 ubuntu ubuntu '
                                                   '175 Jul 25 04:55 '
                                                   'task_agent.md\n'
                                                   '\n'}],
                              'status': 'success',
                              'toolUseId': 'tooluse_1guHDYQ2Su-tAXSLOH3chQ'}}],
  'role': 'user'},
 {'content': [{'text': "'

#### 3.1.2. SummarizingConversationManager

오래된 메시지를 요약하여 중요한 정보를 보존하면서 컨텍스트 한계 내에서 대화를 관리합니다.

**주요 설정:**

| 파라미터 | 타입 | 기본값 | 설명 |
|---------|------|--------|------|
| `summary_ratio` | `float` | `0.3` | 컨텍스트 축소 시 요약할 메시지 비율 (0.1~0.8 범위) |
| `preserve_recent_messages` | `int` | `10` | 항상 유지할 최근 메시지 수 |
| `summarization_agent` | `Agent` | `None` | 요약 생성용 커스텀 에이전트 (system_prompt와 동시 사용 불가) |
| `summarization_system_prompt` | `str` | `None` | 요약용 커스텀 시스템 프롬프트 (agent와 동시 사용 불가) |

> **기본 요약 방식**: 커스텀 설정이 없을 경우, 주요 토픽, 사용된 도구, 기술적 정보를 3인칭 형태의 구조화된 불릿 포인트로 요약합니다.

In [72]:
from strands.agent.conversation_manager import SummarizingConversationManager

In [73]:
# Custom system prompt for technical conversations
custom_system_prompt = """
You are summarizing a technical conversation. Create a concise bullet-point summary that:
- Focuses on code changes, architectural decisions, and technical solutions
- Preserves specific function names, file paths, and configuration details
- Omits conversational elements and focuses on actionable information
- Uses technical terminology appropriate for software development

Format as bullet points without conversational language.
"""

conversation_manager = SummarizingConversationManager(
     summary_ratio=0.3,  # Summarize 30% of messages when context reduction is needed
    preserve_recent_messages=3,  # Always keep 10 most recent messages
    summarization_system_prompt=custom_system_prompt
)

In [74]:
agent.conversation_manager = conversation_manager

In [75]:
message = "안녕 나는 장동진이야"
agent, response = asyncio.run(process_streaming_response(agent, message))
print ("\n")
pprint (agent.messages)

안녕하세요, 장동진님! 다시 만나뵙게 되어 반갑습니다. 오늘도 도움이 필요하신 일이 있으신가요? 어떤 질문이나 작업을 도와드릴까요?

[{'content': [{'text': './prompts 디렉토리에 어떤 파일이 있는지 확인해 줄래?'}], 'role': 'user'},
 {'content': [{'text': "네, './prompts' 디렉토리의 파일 목록을 확인해보겠습니다."},
              {'toolUse': {'input': {'cmd': 'ls -l ./prompts'},
                           'name': 'bash_tool',
                           'toolUseId': 'tooluse_1guHDYQ2Su-tAXSLOH3chQ'}}],
  'role': 'assistant'},
 {'content': [{'toolResult': {'content': [{'text': 'ls -l ./prompts||total 4\n'
                                                   '-rw-rw-r-- 1 ubuntu ubuntu '
                                                   '175 Jul 25 04:55 '
                                                   'task_agent.md\n'
                                                   '\n'}],
                              'status': 'success',
                              'toolUseId': 'tooluse_1guHDYQ2Su-tAXSLOH3chQ'}}],
  'role': 'user'},
 {'content': [{'text': "'./prompts' 디렉토리에는 'task_agent.md